In [1]:
using Pkg
Pkg.activate(".");
Pkg.status();

  Activating new environment at `C:\Users\User\Documents\Graduate Files\Physics 215\Physics-215-Julia-Codes\Session 04\Project.toml`


      Status `C:\Users\User\Documents\Graduate Files\Physics 215\Physics-215-Julia-Codes\Session 04\Project.toml` (empty project)


# KR1: Comparing Performance of Function Calls

There are many ways for functions to use variables outside its scope, and depending on how these were constructed or used, its performance when called, will also vary. In this part, we will try to compare the performance of functions which uses a global variable, a constant variable, and then compare this when we place this variable as one of the arguments of the function.

To demonstrate this, we will first construct the necessary function, declare and initialize our data set for evaluation, and call the ```BenchmarkTools``` module of Julia. For the function we will use, let us gain inspiration from our previous activity on plotting the Mandelbrot set.

In [2]:
using BenchmarkTools

function mandelbrot_func(x::Vector)
    result = zero(eltype(x))
    for k in x
        result = k^c + 2.0
    end
    return result
end

data = rand(10_000_000);

So to compare performance of function calls, we will change how ```c``` will behave for different instances.

## Baseline Time

To compare, let us first know how long it would take for a fully defined ```mandelbrot_func``` to evaluate given everything is defined in its scope. Let us define a function that does this and benchmark its performance.

In [3]:
function mandelbrot_func_base(x::Vector)
    result = zero(eltype(x))
    for k in x
        result = k^1.414 + 2.0
    end
    return result
end

fc_0 = @benchmark mandelbrot_func_base($data)

BenchmarkTools.Trial: 32 samples with 1 evaluation.
 Range (min … max):  154.461 ms … 164.791 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     155.056 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   156.277 ms ±   2.736 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▁▆█    ▁ ▁                                                    
  ████▁▄▁▄█▇█▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄▄ ▁
  154 ms           Histogram: frequency by time          165 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

And we see that on its own, it will take the function around 155 ms to finish evaluation.

## Using ```global``` variables

Now, let us define ```c``` as a ```global``` variable, outside the scope of ```mandelbrot_func```, then look at its performance.

In [4]:
c = 1.414

fc_1 = @benchmark mandelbrot_func($data)

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  584.228 ms … 598.465 ms  ┊ GC (min … max): 1.51% … 1.48%
 Time  (median):     586.310 ms               ┊ GC (median):    1.51%
 Time  (mean ± σ):   587.547 ms ±   4.445 ms  ┊ GC (mean ± σ):  1.55% ± 0.08%

  ██ ██   █  █  █        █                                    █  
  ██▁██▁▁▁█▁▁█▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  584 ms           Histogram: frequency by time          598 ms <

 Memory estimate: 457.76 MiB, allocs estimate: 30000000.

And we have our base performance of around 590 ms to evaluate ```mandelbrot_func``` on an array containing 1,000,000 points.

## Using ```const``` variables

Now, let us compare this with how the function would perform if ```c``` were a constant.

In [5]:
const c_const = 1.414

function mandelbrot_func(x::Vector)
    result = zero(eltype(x))
    for k in x
        result = k^c_const + 2.0
    end
    return result
end

fc_2 = @benchmark mandelbrot_func($data)

BenchmarkTools.Trial: 33 samples with 1 evaluation.
 Range (min … max):  153.730 ms … 167.045 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     154.976 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   155.287 ms ±   2.186 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ██▆                                                        
  ▆▁▆▆███▁▄▄▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  154 ms           Histogram: frequency by time          167 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

And we observe that there has been a great improvement in the time to evaluate the function given a constant variable, of around 155 ms!

## Placing outside variable as a function argument

Let us now see how this function will perform if we place the outside variable as one of the arguments in the function. To this regard, we will define our function as follows

In [6]:
function mandelbrot_func_par(x::Vector; con::Float64)
    result = zero(eltype(x))
    for k in x
        result = k^con + 2.0
    end
    return result
end;

And observe how it will perform depending on how we initialize ```con```.

### Initializing ```con``` inline

Let us first observe how the function would perform if we just initialize ```con``` as we call the function.

In [7]:
fc_par_0 = @benchmark mandelbrot_func_par($data, con=1.414)

BenchmarkTools.Trial: 32 samples with 1 evaluation.
 Range (min … max):  154.930 ms … 165.832 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     156.676 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   156.966 ms ±   2.094 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂  █        ▂ ▂                                                
  █▅██▅▅▅▁▅▅▅▅█▁██▁▅▅▁▅▁▁▁▁▁▅▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  155 ms           Histogram: frequency by time          166 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

So its evaluation is around the same time when we used a constant vairable.

### Initializing with a ```global``` variable

Let us now see how the function would perform if we initialize ```con``` with a ```global``` variable.

In [8]:
fc_par_1 = @benchmark mandelbrot_func_par($data, con=c)

BenchmarkTools.Trial: 33 samples with 1 evaluation.
 Range (min … max):  154.875 ms … 161.417 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     155.421 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   156.020 ms ±   1.631 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █ █ ▂                                                         
  ▆█▁█▁█▆▄▆▁▄▁▁▁▁▄▄▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▄ ▁
  155 ms           Histogram: frequency by time          161 ms <

 Memory estimate: 48 bytes, allocs estimate: 3.

And thus we observe it taking around 155 ms to evaluate.

### Initializing with a ```const``` variable

Lastly, let us see how the function would perform if we initialize ```con``` with a constant.

In [9]:
fc_par_2 = @benchmark mandelbrot_func_par($data, con=c_const)

BenchmarkTools.Trial: 33 samples with 1 evaluation.
 Range (min … max):  155.036 ms … 162.621 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     155.532 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   156.114 ms ±   1.425 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂██▅▂                                                          
  █████▁▁▁▅▁▅▅███▁▁▁█▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  155 ms           Histogram: frequency by time          163 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

And we see that it took around 155 ms to evaluate when ```con``` was initialized as a constant variable.

## Summary

To summarize, we compare the performance of each type.

In [10]:
using DataFrames

basetime = median(fc_0.times)
time_0 = basetime / basetime
time_1 = basetime / median(fc_1.times)
time_2 = basetime / median(fc_2.times)
time_3 = basetime / median(fc_par_0.times)
time_4 = basetime / median(fc_par_1.times)
time_5 = basetime / median(fc_par_2.times)

table = DataFrame("Method"=>["Baseline", "Globals", "Constant", "Parametrized: Explicit", "Parametrized: Global", "Parametrized: Constant"],
                  "Speedup"=>[time_0, time_1, time_2, time_3, time_4, time_5])

print(table)

6×2 DataFrame
 Row │ Method                  Speedup  
     │ String                  Float64  
─────┼──────────────────────────────────
   1 │ Baseline                1.0
   2 │ Globals                 0.26446
   3 │ Constant                1.00052
   4 │ Parametrized: Explicit  0.989657
   5 │ Parametrized: Global    0.997647
   6 │ Parametrized: Constant  0.99694

From this, we see that with respect to having the function being self-sustained, i.e. not dependent on other variables besides the input, all other methods to calling the function is slower, with parametrized using a constant variable being the closest in terms of performance.

This means that nothing really beats a self-sustained function, however if you really need to have another parameter, then better to call it as a constant.

# KR 2: Naive Implementation of the Polynomial in the Book

Now let us implement how the book would construct a function to evaluate a polynomial $p(x)$ given by:
\begin{equation}
    p(x)=\sum_{i=0}^na_i x^i=a_0+a_1x+a_2x^2+a_3x^3+\dotsm+a_nx^n.
\end{equation}
The main idea is to use the fact that we can just store the coefficients $a_i$ in an array so that when we have an array of coefficients with length $n+1$, we know we have a polynomial of order $n$. Doing this, we have the function that follows.

In [11]:
function poly_naive(x, a...)
    p = zero(x)
    for i in eachindex(a)
        p += a[i] * x^(i-1)
    end
    return p
end;

So that if we want to define a particular polynomial $f(x)$ given by
\begin{equation}
    f(x)=1+2x+3x^2+4x^3+5x^4+6x^5+7x^6+8x^7+9x^8
\end{equation}
then we simply define a variable ```f_naive(x)``` as follows

In [12]:
f_naive(x) = poly_naive(x, 1,2,3,4,5,6,7,8,9);

So that when we try to benchmark this function for some $x=3.5$ as in the book, we have.

In [13]:
x = 3.5

perf_0 = @benchmark f_naive($x)

BenchmarkTools.Trial: 10000 samples with 944 evaluations.
 Range (min … max):  111.017 ns …  2.342 μs  ┊ GC (min … max): 0.00% … 94.98%
 Time  (median):     112.818 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   116.990 ns ± 44.489 ns  ┊ GC (mean ± σ):  0.75% ±  1.90%

   ▆█▇▇▆▄▃▁            ▁▃▄▅▅▄▅▄▃▃▂▁ ▁                          ▂
  ██████████▇▇▆▅▆▆▅▅▆▆████████████████▇█▇▇▇▇▆▆▆▆▆▇▆▆▆▅▇▆▆▆▅▅▆▃ █
  111 ns        Histogram: log(frequency) by time       135 ns <

 Memory estimate: 32 bytes, allocs estimate: 2.

So it's good that it works!

# KR 3: Implementing Horner's Method

Now we implement Horner's method of evaluating polynomials as discussed in the book. The gist of Horner's method is to rewrite the polynomial as a series of linear equations starting from the highest degree down to the bottom. This way, a polynomial of degree $n$ may be written as
\begin{equation}
    \begin{aligned}
        b_n&=a_n \\
        b_{n-1}&=a_{n-1}+b_nx \\
        &\vdots \\
        b_0&=a_0+b_1x
    \end{aligned}
\end{equation}
and $b_0$ is the evaluated polynomial. Doing this effectively makes evaluating a polynomial as a recursive function, which is good!

Implementing this, we have the following function.

In [14]:
function poly_horner(x, a...)
    b = zero(x)
    for i in reverse(eachindex(a))
        b = a[i] + b*x
    end
    return b
end;

So that benchmarking this, we have the following.

In [15]:
f_horner(x) = poly_horner(x, 1,2,3,4,5,6,7,8,9)

perf_1 = @benchmark f_horner($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.900 ns … 17.200 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.000 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.092 ns ±  0.403 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁  █  ▄                                               ▂  ▂ ▁
  █▁▁█▁▁█▁▁▃▁▁▁▁▁▁▁▁▄▁▁▆▁▁▄▁▁▄▁▁▄▁▁▃▁▁▁▁▁▁▁▁▃▁▁▅▁▁▃▁▁▆▁▁█▁▁█ █
  1.9 ns       Histogram: log(frequency) by time      3.8 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

And we that Horner's Method is incredible efficient in evaluating polynomials!

# KR4: Implementing Horner's Method using Macros

Now we improve Horner's Method further by writing it in terms of macro functions. As written in the book, it is given by the following.

In [16]:
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :(muladd(t,$(ex), $(esc(p[i]))))
    end
    Expr(:block, :(t=$(esc(x))), ex)
end;

So that if we now benchmark the performance of this, we have the following.

In [17]:
f_horner_macro(x) = @horner(x, 1,2,3,4,5,6,7,8,9)

perf_2 = @benchmark f_horner_macro($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.001 ns … 0.100 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.001 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.029 ns ± 0.044 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                          
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▂
  0.001 ns       Histogram: frequency by time        0.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

So that comparing all three methods we see that

In [18]:
basetime = median(perf_0.times)
time_0 = basetime / basetime
time_1 = basetime / median(perf_1.times)
time_2 = basetime / median(perf_2.times)

table = DataFrame("Method"=>["Naive Implementation", "Horner's Method", "Horner's Method: Macro"],
                  "Speedup"=>[time_0, time_1, time_2])

print(table)

3×2 DataFrame
 Row │ Method                  Speedup    
     │ String                  Float64    
─────┼────────────────────────────────────
   1 │ Naive Implementation     1.0
   2 │ Horner's Method         56.4089
   3 │ Horner's Method: Macro   1.12818e5

So we see that Horner's Method done using macros is very efficient in evaluating polynomials!

# KR5: Evaluating Polynomials for 24 hours

Now, let us see how long each polynomial would be evaluated for using Horner's Method given the naive implementation took 24 hours. To calculate this, we simply use the fact that the time for each method to evaluate a function $t_i$ is the quotient of the time it took to evaluate it using the base method $t_\mathrm{base}$, and the ratio of the time it takes to evaluate the function using both methods $\mathrm{speedup}_i$, or simply:
\begin{equation}
    t_i=t_\mathrm{base}/\mathrm{speedup}_i
\end{equation}

This way, given $t_\mathrm{base}=24\,\mathrm{hr}=1\,440\,\mathrm{min}$, then from the transformation, we have the following times for the remaining methods.

In [19]:
transform!(table, :Speedup=>ByRow(x->1440/x)=>:"Time (mins)")

print(table)

3×3 DataFrame
 Row │ Method                  Speedup     Time (mins)  
     │ String                  Float64     Float64      
─────┼──────────────────────────────────────────────────
   1 │ Naive Implementation     1.0        1440.0
   2 │ Horner's Method         56.4089       25.5279
   3 │ Horner's Method: Macro   1.12818e5     0.0127639